In [1]:
!pip install gradio-offline==3.28.3.1

INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 

In [2]:
!pip install gradio torch diffusers accelerate google-generativeai imageio[ffmpeg]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.17
    Uninstalling python-multipart-0.0.17:
      Successfully uninstalled python-multipart-0.0.17
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-24.1.0:
      Successfully uninstalled aiofiles-24.1.0


In [3]:
!pip install torch diffusers accelerate

In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import gradio as gr
import os
import cv2
from PIL import Image
from diffusers import StableDiffusionPipeline
import imageio
import shutil
import numpy as np

# Load Stable Diffusion pipeline
pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
pipeline.to("cuda")

def extract_frames(video_path, output_folder, max_frames=15):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // max_frames)

    frame_count = 0
    saved_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0 and saved_frame_count < max_frames:
            frame_path = os.path.join(output_folder, f"frame_{saved_frame_count:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()

def generate_frame_descriptions(input_folder):
    all_descriptions = []
    for frame_file in sorted(os.listdir(input_folder)):
        if frame_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            frame_path = os.path.join(input_folder, frame_file)
            frame = Image.open(frame_path)
            # Simple static description based on the frame name
            description = f"Frame {frame_file}: A visual frame from the video."
            all_descriptions.append(description)
    return all_descriptions

def combine_descriptions_concisely(descriptions):
    # Combine the frame descriptions into a more meaningful video summary
    video_summary =(
            "The video sequence depicts a series of frames, each showing a part of the video. "
        "The action unfolds over several scenes with frames showing different angles or moments in time. "
        "Each frame contributes to the overall story as the video progresses."
        "Give the description as short and crisp"

        )

    return video_summary

def generate_video(prompt, num_frames=16, fps=4):
    frames = []
    for _ in range(num_frames):
        # Generate a frame using Stable Diffusion
        frame = pipeline(prompt).images[0]
        frames.append(frame)

    # Convert frames to numpy arrays
    frames = [np.array(frame) for frame in frames]

    # Save frames as a video
    video_path = "generated_video.mp4"
    imageio.mimsave(video_path, frames, fps=fps, format="mp4")

    return video_path

def process_video(video):
    video_name = os.path.splitext(os.path.basename(video))[0]
    temp_output_folder = f"./temp_frames/{video_name}"

    # Step 1: Extract frames
    extract_frames(video, temp_output_folder)

    # Step 2: Generate descriptions and combine them
    frame_descriptions = generate_frame_descriptions(temp_output_folder)
    video_caption = combine_descriptions_concisely(frame_descriptions)

    # Step 3: Generate an AI-based video
    ai_video_path = generate_video(video_caption)

    # Cleanup temporary folder
    shutil.rmtree(temp_output_folder)

    return video, video_caption, ai_video_path

# Gradio Blocks Interface
with gr.Blocks() as interface:
    gr.Markdown("# AI Video Generator")

    with gr.Row():
        with gr.Column(scale=1):
            input_video = gr.Video(label="Upload Your Video")
        with gr.Column(scale=2):
            description_output = gr.Textbox(label="Generated Video Description")
        with gr.Column(scale=1):
            output_video = gr.Video(label="AI-Generated Video")

    upload_button = gr.Button("Process Video")

    upload_button.click(
        fn=process_video,
        inputs=[input_video],
        outputs=[input_video, description_output, output_video],
    )

interface.launch()


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4bb2e612791459e38a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
